<a href="https://colab.research.google.com/github/satvamthakkar/blooms-questions/blob/main/Question_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.2 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which

In [ ]:
import os
import transformers
import torch
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer

2024-04-01 21:13:14.294139: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 21:13:14.294233: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 21:13:14.424470: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
HF_TOKEN = "your_api_key"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=HF_TOKEN)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=HF_TOKEN)

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

# Creating Dataset

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/kaggle/input/blooms-taxonomy-dataset/B.csv")

In [ ]:
df

,Question,Label
0,Can you list the ingredients needed for a clas...,Remember
1,What are the top five tourist attractions in P...,Remember
2,List five common programming languages.,Remember
3,Name three types of renewable energy sources.,Remember
4,Can you list the planets in our solar system?,Remember
...,...,...
1766,Can you generalize from historical economic cy...,Create
1767,How can you relate knowledge from history and ...,Create
1768,What conclusions can you draw by combining int...,Create
1769,How could old architectural styles be combined...,Create


In [ ]:
!pip install keybert

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 5.0 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.4-py3-none-any.whl size=39199 sha256=befc7190707738b124e86d514f6eeac47a123f329d5990e295ea1410622ba5e5
  Stored in directory: /root/.cache/pip/wheels/97/ef/4c/6588bd7072b0cc04225b40e639b991e49ebd4e21fb81f0acee
Successfully built keybert


In [ ]:
from keybert import KeyBERT

kw_model = KeyBERT()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
keywords = kw_model.extract_keywords(df['Question'][0])

In [ ]:
keywords[0][0]

'spaghetti'

In [ ]:
keyword = []

for data in df['Question']:
    keyword.append(kw_model.extract_keywords(data)[0][0])

In [ ]:
keyword[:5]

['spaghetti', 'attractions', 'languages', 'renewable', 'planets']

In [ ]:
df["Keyword"] = keyword

In [ ]:
df

,Question,Label,Keyword
0,Can you list the ingredients needed for a clas...,Remember,spaghetti
1,What are the top five tourist attractions in P...,Remember,attractions
2,List five common programming languages.,Remember,languages
3,Name three types of renewable energy sources.,Remember,renewable
4,Can you list the planets in our solar system?,Remember,planets
...,...,...,...
1766,Can you generalize from historical economic cy...,Create,predict
1767,How can you relate knowledge from history and ...,Create,cultural
1768,What conclusions can you draw by combining int...,Create,interpretations
1769,How could old architectural styles be combined...,Create,buildings


In [ ]:
print()

# Prior to Fine Tuning

In [ ]:
text = f'Generate a Question related only on the Keyword - "{df["Keyword"][0]}" {context} and Bloom Taxonomy Type - "{df["Label"][0]}" {pages} {articles} \nGenerated Question: \nAnswer:'
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Generate a Question related only on the Keyword - "spaghetti" and Bloom Taxonomy Type - "Remember" 
Generated Question: 
What is the difference between a spaghetti and a spaghetti sauce? 
Answer: 
A


In [ ]:
os.environ["WANDB_DISABLED"] = "false"

In [ ]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [ ]:
def formatting_func(example):
    text = f'Generate a Question related only on the Keyword - "{example["Keyword"]}" and Bloom Taxonomy Type - "{example["Label"]}" \nGenerated Question: {example["Question"]}'
    return [text]

In [ ]:
from datasets import Dataset

hf_dataset = Dataset.from_pandas(df)

In [ ]:
formatting_func(hf_dataset[1])

['Generate a Question related only on the Keyword - "attractions" and Bloom Taxonomy Type - "Remember" \nGenerated Question: What are the top five tourist attractions in Paris?']

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=hf_dataset,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=50,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

Map:   0%|          | 0/1771 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
import wandb

wandb.login(key="your_api_key")

wandb.init(project='your_project', entity='your_entity')

wandb: Currently logged in as: satvamthakkar1 (ipdproject). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: satvamthakkar1. Use `wandb login --relogin` to force relogin


In [ ]:
trainer.train()

Step,Training Loss
1,1.328100
2,1.328100
3,1.318400
4,1.296100
5,1.273900
6,1.254000
7,1.238500
8,1.223000
9,1.202700
10,1.180100


TrainOutput(global_step=50, training_loss=0.8448035389184951, metrics={'train_runtime': 215.2721, 'train_samples_per_second': 0.929, 'train_steps_per_second': 0.232, 'total_flos': 1223694483456000.0, 'train_loss': 0.8448035389184951, 'epoch': 50.0})

# Post Finetuning

In [ ]:
text = f'Generate a Question related only on the Keyword - "react" and Bloom Taxonomy Type - "Remember" \nGenerated Question: '
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Generate a Question related only on the Keyword - "react" and Bloom Taxonomy Type - "Remember" 
Generated Question: 
What is the difference between a React component and a React function? 
Answer: 



In [ ]:
text = f'Generate a Question related only on the Keyword - "react" and Bloom Taxonomy Type - "Understand" \nGenerated Question: '
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Generate a Question related only on the Keyword - "react" and Bloom Taxonomy Type - "Understand" 
Generated Question: 
Explain the difference between the following two terms: 
1. React and Redux 
2


In [ ]:
text = f'Generate a Question related only on the Keyword - "react" and Bloom Taxonomy Type - "Apply" \nGenerated Question: '
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Generate a Question related only on the Keyword - "react" and Bloom Taxonomy Type - "Apply" 
Generated Question: 
How to use the React Hooks?
 
 
 
 
 
 


In [ ]:
text = f'Generate an MCQ Question related only on the Keyword - "react" and Bloom Taxonomy Type - "Analyze" \nGenerated Question: '
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Generate a Question related only on the Keyword - "react" and Bloom Taxonomy Type - "Analyze" 
Generated Question: 
How to use the React library?
Answer: 
React is a JavaScript library for building


In [ ]:
text = f'Generate a Question related only on the Keyword - "react" and Bloom Taxonomy Type - "Evaluate" \nGenerated Question: '
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Generate a Question related only on the Keyword - "react" and Bloom Taxonomy Type - "Evaluate" 
Generated Question: 
How to use the React Hooks?
 
Generated Answer: 
React Hooks are a


In [ ]:
text = f'Generate a Question related only on the Keyword - "react" and Bloom Taxonomy Type - "Create" \nGenerated Question: '
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Generate a Question related only on the Keyword - "react" and Bloom Taxonomy Type - "Create" 
Generated Question: 
How to use the React Hooks?
 
 
 
 
 
 
